Попробовала дообучить rubert-tiny2 на датасете blinoff/medical_qa_ru_data.
Обучать на CPU это боль)) в колабе не смогла дождаться, обучение прервалось через 6 часов. Сократила до 2-х эпох и урезала датасет. Доработала функцию предобработки текста. Лематизацию пришлось закомментрировать ввиду длительности процесса.   использовала импорт модели с модулем Взяла всего 20%, передала на cuda, дообучение модели занимает порядка 40-50 минут.  Дообученная модель выдает слишком короткие ответы.

Модель cointegrated/rubert-tiny2 является русскоязычной моделью языковой модели BERT, обученной на небольшом размере данных. Вот подробная характеристика этой модели:

- Название модели: cointegrated/rubert-tiny2
- Базовая модель: RuBERT (Russian BERT)
- Размер модели: Текущий размер модели составляет примерно 42,7 МБ.
- Конфигурация модели: Это модель BERT, предназначенная для генерации условных последовательностей. Она может использоваться для задач, таких как текстовая генерация, ответы на вопросы, перевод и другие задачи, где требуется генерировать последовательности текста на основе окружающего контекста.

Для дообучения модели для вопросно-ответной системы для чат-бота используется модуль AutoModelForCausalLM, который является частью библиотеки transformers и предоставляет предобученные модели для задачи генерации условных последовательностей (Causal Language Modeling).

AutoModelForCausalLM:

1. Модуль AutoModelForCausalLM предоставляет общий интерфейс для работы с различными моделями генерации условных последовательностей. Он может автоматически выбирать соответствующую модель, основываясь на названии модели, указанном в качестве параметра.

2. Обеспечивает поддержку различных архитектур: может использоваться с различными предобученными архитектурами моделей, такими как BERT, GPT, GPT-2, XLNet и другими, что позволяет выбрать подходящую модель для конкретной задачи генерации текста.

3. Использование предобученных весов: Модуль AutoModelForCausalLM загружает предобученные веса модели, которые позволяют использовать модель для генерации текста с уже полученными знаниями о языке.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install accelerate -U

In [3]:
!pip install transformers sentencepiece

In [4]:
! wget https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv

! pip install corus

--2023-09-21 04:42:26--  https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv
Resolving huggingface.co (huggingface.co)... 65.8.178.27, 65.8.178.118, 65.8.178.93, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.27|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/blinoff/medical_qa_ru_data/6aca4fb6dae499745c9ca1dfad8a0ade809573daf228c923fdf846ba4a639b50?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27medical_qa_ru_data.csv%3B+filename%3D%22medical_qa_ru_data.csv%22%3B&response-content-type=text%2Fcsv&Expires=1695530547&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NTUzMDU0N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9ibGlub2ZmL21lZGljYWxfcWFfcnVfZGF0YS82YWNhNGZiNmRhZTQ5OTc0NWM5Y2ExZGZhZDhhMGFkZTgwOTU3M2RhZjIyOGM5MjNmZGY4NDZiYTRhNjM5YjUwP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249

In [5]:
!pip install datasets transformers==4.28.0

In [6]:
!pip install pymorphy2
!pip install re
!pip install nltk
!pip install unicodedata
!pip install contractions
!pip install inflect
!pip install emoji

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata


In [7]:
!pip install num2words

In [8]:
path_m =  "/content/drive/MyDrive/Data Science_models/nlp/project/"

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
import string
import pymorphy2
import re

import emoji
import unicodedata
import contractions
import inflect
from num2words import num2words


In [10]:
nltk.download('punkt')  # Токенизатор
nltk.download('stopwords')  # Стоп-слова
nltk.download('wordnet')  # Лемматизатор

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import torch
#from transformers import AutoTokenizer, AutoModelForQuestionAnswering
#from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split



In [12]:
# Загрузка набора данных Medical QA Ru Data
dataset = load_dataset('blinoff/medical_qa_ru_data', split= 'train[:10%]')



In [13]:
# Преобразование Dataset в список словарей
dataset_dict = dataset.to_dict()

# Преобразование списка словарей в объект Pandas DataFrame
data = pd.DataFrame(dataset_dict, columns = ['desc', 'ans'])

# Просмотр первых 5 строк датафрейма
print(data.head(5))

                                                desc  \
0  Ларипронт 20 талеток,через каждые 2-3 часа.Оче...   
1  Здравствуйте, я на 7-8 неделе беременности. С ...   
2  Здравствуйте месячные должны придти 23 марта в...   
3  Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...   
4  Мне прописали пить Аллохол. Врач написала пить...   

                                                 ans  
0  Что вы им лечите? Длительность приема Ларипрон...  
1  Здравствуйте, это может быть признаком раннего...  
2                         Выполните исследование хгч  
3  Можно.;\nЗдравствуйте , да, попейте сладкого ч...  
4  Препарат принимается после еды. Уточните это  ...  


In [14]:
dataset['desc'][0]

'Ларипронт 20 талеток,через каждые 2-3 часа.Очень понравились,но пока принимала,чувствовала себя хорошо. Закончились-все сиптомы опять проявились. Нигде не найду как долго их можно принимать. Скажите пожалуйста.'

In [15]:
dataset['ans'][0]

'Что вы им лечите? Длительность приема Ларипронта индивидуальна и устанавливается лечащим врачом.;\nУточните у своего лечащего врача.'

In [16]:
dataset['spec10'][0]

'Отоларинголог'

In [17]:
train_q, test_q, train_ans, test_ans = train_test_split(data.desc,
                                                        data.ans,
                                                        test_size=.2,
                                                        random_state=42)

In [18]:
device = torch.device("cuda")

Модуль AutoModelForCausalLM и модуль AutoModelForQuestionAnswering являются двумя разными модулями, предназначенными для различных задач.

Модуль AutoModelForCausalLM предназначен для создания автоматически настраиваемой модели для генерации последовательностей, где последовательности имеют причинно-следственную связь. Например, такая модель может использоваться для генерации продолжения рассказа или прогнозирования следующего слова в текстовой последовательности.

С другой стороны, модуль AutoModelForQuestionAnswering предназначен для создания автоматически настраиваемой модели для ответа на вопросы на основе контекстных данных. Например, такая модель может использоваться для отвечения на вопросы в рамках задачи вопросно-ответной системы, в которой нужно найти ответ на вопрос в текстовом документе или корпусе данных.

In [19]:
#model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
#tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
#model = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2').to(device)

In [20]:

# Инициализация токенизатора и модели BERT
#tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
#model = AutoModelForCausalLM.from_pretrained("cointegrated/rubert-tiny2").to(device)
#model = AutoModelForQuestionAnswering.from_pretrained("cointegrated/rubert-tiny2").to(device)
#model.cuda()  # uncomment it if you have a GPU
#model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [21]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
# Функция для очистки текста из статьи https://habr.com/ru/articles/738176/ адаптированная под русский язык
def preproc_text(text):
    """Очищает текст от HTML-тегов, URL-ссылок, эмоджи, приводит к нижнему регистру,
    удаляет специальные символы, удаляет стоп-слова,
    лемматизирует слова и удаляет знаки препинания.

    Args:
        input_text (str): Входной текст для очистки.

    Returns:
        str: Очищенный текст.
    """

    res = str(text).strip()

    # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
    res = re.sub('<[^<]+?>', '', res)

    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    res = re.sub(r'http\S+', '', res)

    # Эмоджи и эмотиконы: используем собственную функцию для преобразования эмоджи в текст
    res = emojis_words(res)

    # Приводим все входные данные к нижнему регистру
    res = res.lower()

    # Убираем лишние пробелы
    res = re.sub(r"\s+", " ", res)

    # Убираем специальные символы: избавляемся от всего, что не является "словами"
    res = re.sub('[^а-яА-ЯёЁ0-9\s]', '', res)

    # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
    stop_words = set(stopwords.words('russian'))
    words = res.split()
    words = [word for word in words if word not in stop_words]
    res = ' '.join(words)

    # Лемматизация слов
    #morph = pymorphy2.MorphAnalyzer()
   # lemmas = [morph.parse(word)[0].normal_form for word in words] # этот процесс сильно замедляет обработку текста (3 часа на 20% данного датасета)
    #res = ' '.join(lemmas)
    # Знаки препинания: далее - удаляем из текста все знаки препинания
    res = re.sub(r'[^\w\s]', '', res)

    return res

# Функция для преобразования эмоджи в слова
def emojis_words(text):

    # Модуль emoji: преобразование эмоджи в их словесные описания
    clean_text = emoji.demojize(text, delimiters=(" ", " "))

    # Редактирование текста путём замены ":" и" _", а так же - путём добавления пробела между отдельными словами
    clean_text = clean_text.replace(":", "").replace("_", " ")

    return clean_text


def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = preproc_text(texts)
        answer = preproc_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data

In [23]:
#train_data = build_data(train_q, train_ans)
#test_data = build_data(test_q, test_ans)

In [24]:
from tqdm import tqdm

# Форматирование тренировочных данных с прогресс-баром
train_data = []
for idx, texts in tqdm(enumerate(train_q), total=len(train_q), desc='Processing train data'):
    question = preproc_text(texts)
    answer = preproc_text(train_ans.iloc[idx])
    res = '\nx:' + question + '\ny:' + answer
    train_data.append(res)

# Форматирование тестовых данных с прогресс-баром
test_data = []
for idx, texts in tqdm(enumerate(test_q), total=len(test_q), desc='Processing test data'):
    question = preproc_text(texts)
    answer = preproc_text(test_ans.iloc[idx])
    res = '\nx:' + question + '\ny:' + answer
    test_data.append(res)

Processing test data: 100%|██████████| 3807/3807 [00:04<00:00, 784.35it/s]


Этот код устанавливает токен, который будет использоваться при заполнении (padding) последовательностей текстовых данных при обучении нейронных сетей. Это делается с помощью установки значения eos_token (токена конца последовательности) как значения pad_token (токена-заполнителя).

Обычно в обучающем наборе данных бывают последовательности, которые имеют разную длину. Для обеспечения удобства и эффективности обучения модели, все последовательности должны быть одинаковой длины. Один из подходов к достижению этой цели - добавление токенов-заполнителей в конец последовательности до тех пор, пока ее длина не станет равной максимальной длине в обучающем наборе данных.

Установка pad_token в значение eos_token означает, что при заполнении последовательности токеном-заполнителем будет использоваться токен, обозначающий конец последовательности. Это может быть полезно, когда необходимо предотвратить модель от генерации ненужной информации после конца последовательности, ведь такие лишние токены могут негативно сказаться на точности модели.

In [25]:
tokenizer.pad_token = tokenizer.eos_token

In [26]:
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Токенизируем тренировочный и тестовый датасеты:

In [27]:
train_encodings = tokenizer(train_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

In [28]:
class QAMedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = QAMedicalDataset(train_encodings)
test_dataset = QAMedicalDataset(test_encodings)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [29]:
# Настройка тренировочных аргументов
training_args = TrainingArguments(
    output_dir="./models", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

# Задание значения max_steps перед инициализацией тренера
#training_args.max_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs

# Определение тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=test_dataset
)




In [30]:
# Обучение модели
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-28-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,4.847900
1000,4.508200
1500,4.365600
2000,4.330700
2500,4.249300
3000,4.198700
3500,4.189200
4000,4.031000
4500,3.874600
5000,3.867100


<ipython-input-28-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-28-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-28-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-28-b29274360c54>:6: UserWarning: To copy construct from a 

TrainOutput(global_step=11421, training_loss=3.9335840557723984, metrics={'train_runtime': 6968.6398, 'train_samples_per_second': 6.555, 'train_steps_per_second': 1.639, 'total_flos': 1.1936081313792e+16, 'train_loss': 3.9335840557723984, 'epoch': 3.0})

Сохраним модель на диск:

In [33]:
tokenizer.save_pretrained(path_m + 'tokenizer_qa_medical_up_3')
#model.save_pretrained(path_m + 'model_qa_medical_up_1')
model.save_pretrained(path_m + 'model_qa_medical_up_3')

In [34]:
tokenizer = AutoTokenizer.from_pretrained(path_m + 'tokenizer_qa_medical_up_3')
model = AutoModelForCausalLM.from_pretrained(path_m + 'model_qa_medical_up_3')

Функция для ведения диалога. Модель "держит в голове" контекст последних 10-ти сообщений.

In [35]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        eos_token_id=end_token_id,
        do_sample=True,
        max_length=size+128,
        repetition_penalty=3.2,
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = re.findall(r'\ny:(.+)', decoded)[-1]
    return result.strip()

Протестируем генератор:

In [36]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'Стоп':
        break
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
как лечить горло
здравствуйте орошать мирамистином полоскать фурациллиномсмазывать хлоргексидиномпринимать антибиотик пользователь поблагодарил ответ сумму 10 бонусов текст благодарности спасибо консультацию можете орошать мирамистином полоскание горла раствором соды рассасывания лизобакта 1т3р рать 7дн ингалипт 2в3р горло закапывать борную кислоту 3 раза день температура тела должна повышаться выше 385 372 принимать жаропонижающие препараты обратитесь терапевту очном порядкене занимайтесь самолечением здравствуйте начните циклоферон парацетам
что делать, если болит нога
выполните снимок сустава покажитесь травматологу выполните рентген мрт суставапокажитесь ортопеду нужно исключить перелом голеностопного сустава возможно развитие артрита ревматоидного артритавам нужно выполнить рентгенографию сустава показаться травматологувозможен остеохондроз межпозвонкового дискаисключите патологию поясничнокрестцового отдела позвоночникадля назначения лечения об

Результат далек от совершенства, но лучше, чем с минимальной предобработкой текста. Явный недостаток- модель не разделяет некоторые слова.